In [25]:
import pandas as pd
import numpy as np
import collections

In [26]:
# This dataframe creates vectorized season data for each team

all_teams_pd = pd.read_csv('ncaam-march-mania-2021/MDataFiles_Stage1/MTeams.csv')
all_teams_list = all_teams_pd['Team_Name'].tolist()

def season_vector(team_id, year):
    return df.query('WTeamID == {} and Season == {}'.format(team_id, year)).values.tolist()

def compare_teams(team_id_1, team_id_2, year):
    team_1 = season_vector(team_id_1, year)
    team_2 = season_vector(team_id_2, year)
    comparison = [a - b for a, b in zip(team_1, team_2)]
    return comparison

def all_teams_season(year):
    season_dictionary = collections.defaultdict(list)
    for team in all_teams_list:
        team_id = all_teams_pd[all_teams_pd['Team_Name'] == team].values[0][0]
        team_vector = season_vector(team_id, year)
        season_dictionary[team_id] = team_vector
    return season_dictionary

def codify_home_away(row_num):
    if (row_num == 'H'):
        home = 1
    if (row_num == 'A'):
        home = -1
    if (row_num == 'N'):
        home = 0
    return home

TypeError: unsupported operand type(s) for -: 'list' and 'list'

In [ ]:
def create_training_set(year_list):
    num_games = 0
    for year in year_list:
        season = regseason_compact_results_df[regseason_compact_results_df['Season'] == year]
        num_games += len(season.index)
        tourney = tourney_compact_results_df[tourney_compact_results_df['Season'] == year]
        num_games += len(tourney.index)
    num_features = len(season_vector(1228, 2018))
    x_train = np.zeros((num_games, num_features + 1))
    y_train = np.zeros((num_games))

    i = 0
    for year in year_list:
        team_vector = all_teams_season(year)
        season = regseason_compact_results_df[regseason_compact_results_df['Season'] == year]
        num_games_in_season = len(season.index)
        tourney = tourney_compact_results_df[tourney_compact_results_df['Season'] == year]
        num_games_in_season += len(tourney.index)
        x_train_season = np.zeros((num_games_in_season, num_features + 1))
        y_train_season = np.zeros((num_games_in_season))
        j = 0
        for i, row in season.iterrows():
            w_team = row['WTeamID']
            w_team_vector = team_vector[w_team]
            l_team = row['LTeamID']
            l_team_vector = team_vector[l_team]
            comparison = [a - b for a, b in zip(w_team_vector, l_team_vector)]
            loc = codify_home_away(row['WLoc'])
            if j % 2 == 0:
                comparison.append(loc)
                x_train_season[j] = comparison
                y_train_season[j] = 1
            else:
                comparison.append(-loc)
                x_train_season[j] = [ -p for p in comparison]
                y_train_season[j] = 0
            j += 1
        for i, row in tourney.iterrows():
            w_team = row['WTeamID']
            w_team_vector = team_vector[w_team]
            l_team = row['LTeamID']
            l_vector = team_vector[l_team]
            comparison = [a - b for a, b in zip(w_team_vector, l_team_vector)]
            loc = 0 #All tournament games are neutral
            if (j % 2 == 0):
                comparison.append(loc)
                x_train_season[j] = comparison
                y_train_season[j] = 1
            else:
                comparison.append(-loc)
                x_train_season[j] = [ -p for p in comparison]
                y_train_season[j] = 0
            j += 1
        x_train[i:num_games_in_season+i] = x_train_season
        y_train[i:num_games_in_season+i] = y_train_season
        i += num_games_in_season
    return x_train, y_train

In [ ]:
# Add standardization here

In [ ]:
years = range(1993,2017)
x_train, y_train = create_training_set(years)
np.save('xTrain', x_train)
np.save('yTrain', y_train)